# Tutorial: Data Primitives

Adapted from "Introduction to the Data Primitives" in the Fermi GBM Data Tools documentation and "Data Primitives" in the Gamma-ray Data Tools Core Package documentation.

Data primitives are the data classes that define the datatypes within the GDT at the most basic level. They handle the properties and manipulation of the fundamental types of data that GBM produces. Often you won’t need to directly worry about these classes and how they work if you are doing sufficiently high-level work using the rest of the data tools (e.g. plotting lightcurves/spectra, standard data binning, background/spectral fitting, etc.); however, the API exposes a full array of properties and methods that allow you to work directly with the data primitives.

In this tutorial, we will go through multiple examples of how to understand and manipulate the data primitives used in the Fermi Gamma-Ray Tools (GDT) toolkit and the core GDT toolkit.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gdt.missions.fermi
import gdt.core

## Range and Intervals

A value covering a defined interval, such as time interval or energy range, can be represented by a `Range`. Two specific types of `Range` objects are provided: `EnergyRange` and `TimeRange`. A series of `Range` objects are represented by an `Intervals` object. Similar to the energy- and time-specific `Range` classes, there are energy- and time-specific `Intervals` classes: `Ebounds` and `Gti`, respectively. 

To create a `Range` object, you simply need to supply the starting and ending values. 

In [2]:
from gdt.core.data_primitives import EnergyRange, TimeRange

# create EnergyRange object with range of 50.0 to 300.0
erange = EnergyRange(50.0, 300.0)
print(erange)

# create TimeRange object with range of -10.0 to 10.0
trange = TimeRange(-10.0, 10.0)
print(trange)

<EnergyRange: (50.0, 300.0)>
<TimeRange: (-10.0, 10.0)>


After creating a `Range`, you can easily access properties such as the `Range` object's total width and center value. 

In [71]:
# return width and center of the EnergyRange
print('EnergyRange width:', erange.width)
print('EnergyRange center:', erange.center)

# return width and center of TimeRange
print('TimeRange width:', trange.width)
print('TimeRange center:', trange.center)

EnergyRange width: 250.0
EnergyRange center: 175.0
TimeRange width: 20.0
TimeRange center: 0.0


You can check to see if the `Range` contains a particular value by using the `contains()` method. By default, if the value you wish to check is one of the endpoints of the `Range`, the `contains()` method will return True (that the value **is** included in the `Range`). However, if you wish for the endpoints of the `Range` to not be marked as included, you can set the `inconclusive` parameter in the method to equal False.

In [4]:
# determine if EnergyRange contains value 100.0
print('EnergyRange contains 100.0:', erange.contains(100.0))

# determine if TimeRange contains value 10.0
print('TimeRange contains 10.0:', trange.contains(10.0))

# determine if TimeRange contains value 10.0, changing method to exclude edges of the range for the check
print('TimeRange contains 10.0:', trange.contains(10.0, inclusive=False))

EnergyRange contains 100.0: True
TimeRange contains 10.0: True
TimeRange contains 10.0: False


You can also do simple `Range` “arithmetic”. The intersection or union of two `Range` objects can also be computed to return a new `Range`.

In [5]:
# create second EnergyRange
erange2 = EnergyRange(250.0, 500.0)

# return a new EnergyRange that is the intersection of the first and second EnergyRange objects
print(EnergyRange.intersection(erange, erange2))

# create a second TimeRange
trange2 = TimeRange(30.0, 100.0)

# return a new TimeRange that is the union of the first and second TimeRange objects
print(TimeRange.union(trange, trange2))

<EnergyRange: (250.0, 300.0)>
<TimeRange: (-10.0, 100.0)>


Often one is working with a series of ranges, or `Intervals`. As mentioned previously, the `Gti` class, (Good Time Intervals) is used for a set of time intervals, and the `Ebounds` class is used for a set of energy intervals. There are a few ways to create an `Intervals` object. For example, you could easily initialize an `Intervals` object with a preexisting `Range` object. You could also create an `Intervals` object from a list of lower and upper bounds using the `from_bounds()` method, or you could create the object from a list of lower/upper tuples using the `from_list()` method.

In [6]:
from gdt.core.data_primitives import Ebounds, Gti

# create Ebounds object with 1 interval of our first EnergyRange object
ebounds1 = Ebounds(erange)
print(ebounds1)

# create first Gti object with 3 intervals from a list of 3 sets of lower and upper bounds 
gti1 = Gti.from_bounds((-10.0, 0.0, 10.0), (-5.0, 5.0, 15.0))
print(gti1)

# create second Gti object with 2 intervals from a list of 2 tuples 
gti2 = Gti.from_list([(20.0, 25.0), (50.0, 55.0)])
print(gti2)

<Ebounds: 1 intervals; range (50.0, 300.0)>
<Gti: 3 intervals; range (-10.0, 15.0)>
<Gti: 2 intervals; range (20.0, 55.0)>


Once initialized, the ranges within the `Intervals` object can be retrieved as a list or individually, alongside other properties such as the number of distinct intervals within the object and the object's total range. 

In [7]:
# print all intervals of Gti object
print('GTI intervals:', gti1.intervals)

# print only second interval of Gti object
print('GTI 2nd interval:', gti1[1])

# print the number of intervals in the Gti object
print('GTI number of intervals:', gti1.num_intervals)

# print the total range of the Gti object
print('GTI range:', gti1.range)

GTI intervals: [<TimeRange: (-10.0, -5.0)>, <TimeRange: (0.0, 5.0)>, <TimeRange: (10.0, 15.0)>]
GTI 2nd interval: <TimeRange: (0.0, 5.0)>
GTI number of intervals: 3
GTI range: (-10.0, 15.0)


Similar to the `Range` classes, you can check if a value is contained in the `Intervals` using the `contains()` method. 

In [8]:
# determine if Gti object contains value 5.0
print('GTI contains 5.0:', gti1.contains(5.0))

# determine if Gti object contains value 6.0
print('GTI contains 6.0:', gti1.contains(6.0))

GTI contains 5.0: True
GTI contains 6.0: False


A new `Range` can be inserted into the `Intervals` using the `insert()` method. 

In [9]:
# insert new interval into Gti object
gti1.insert(TimeRange(7.0, 8.0))

# print all intervals, number of intervals, and total range of Gti object with newly inserted interval
print('GTI intervals:', gti1.intervals)
print('GTI number of intervals:', gti1.num_intervals)
print('GTI range:', gti1.range)

GTI intervals: [<TimeRange: (-10.0, -5.0)>, <TimeRange: (0.0, 5.0)>, <TimeRange: (7.0, 8.0)>, <TimeRange: (10.0, 15.0)>]
GTI number of intervals: 4
GTI range: (-10.0, 15.0)


Two `Intervals` can be merged into one using the `merge()` method. 

In [10]:
# merge together first and second Gti objects to create new Gti object
gti_merged = Gti.merge(gti1, gti2)
print(gti_merged)

<Gti: 6 intervals; range (-10.0, 55.0)>


If you want to see the full `Intervals` object, it can easily be printed with each individual range as a list using the `as_list()` method. 

In [11]:
# display all intervals for merged Gti object as list
print(gti_merged.as_list())

[(-10.0, -5.0), (0.0, 5.0), (7.0, 8.0), (10.0, 15.0), (20.0, 25.0), (50.0, 55.0)]


Finally, we can perform the intersection of two `Intervals` objects using the same method that we did for the `Range` objects. 

In [12]:
# create third Gti object
gti3 = Gti(TimeRange(4.0, 12.0))

# return new Gti object that is the intersection of the merged Gti object and the third Gti object
gti_intersect = Gti.intersection(gti_merged, gti3)

# print all intervals in intersection Gti object
print(gti_intersect.intervals)

[<TimeRange: (4.0, 5.0)>, <TimeRange: (10.0, 12.0)>]


## 1D Binned Data

A set of intervals with a corresponding value for each interval, such as the number of items in a series of bins, can be represented by a `Bins` object. This is essentially a type of 1D histogram with a variety of useful properties and methods. A useful, specialized histogram for containing gamma-ray data is defined by the `ExposureBins` class. This class modifies `Bins` by incorporating an exposure for each bin. Furthermore, distinct `ChannelBins`, `EnergyBins`, and `TimeBins` classes are used to contain spectral and temporal data. 

To create a `TimeBins` or `EnergyBins` object, you need to initialize with an array of counts in each bin, the lower bin edges, upper bin edges, and the exposure for each bin.

In [13]:
from gdt.core.data_primitives import TimeBins, EnergyBins
# create TimeBins object with listed counts, edge boundaries, and exposures
counts = [1, 2, 3, 2, 1, 0]
low_edges = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
high_edges = [0.2, 0.3, 0.35, 0.5, 0.6, 0.7]
exposure = [0.08, 0.08, 0.04, 0.08, 0.08, 0.08]
time_bins = TimeBins(counts, low_edges, high_edges, exposure)
print(time_bins)

<TimeBins: 6 bins;
 range (0.1, 0.7);
 2 contiguous segments>


Notice in this example that there is a gap between the bins resulting in two distinct, contiguous segments. Those segments can be directly accessed using the `contiguous_bins()` method.

In [14]:
# print both distinct segments of the TimeBins object
print(time_bins.contiguous_bins())

# print centroids and counts for each TimeBins segment
print('Centroids, counts for each segment:', [(tb.centroids, tb.counts) for tb in time_bins.contiguous_bins()])

[<TimeBins: 3 bins;
 range (0.1, 0.35);
 1 contiguous segments>, <TimeBins: 3 bins;
 range (0.4, 0.7);
 1 contiguous segments>]
Centroids, counts for each segment: [(array([0.15 , 0.25 , 0.325]), array([1, 2, 3])), (array([0.45, 0.55, 0.65]), array([2, 1, 0]))]


A variety of properties are available, such as the rate, uncertainty, bin centroids, and total range of the object:

In [15]:
# return count rates, count rate uncertainties, centroids, and range of TimeBins object
print('TimeBins rates:', time_bins.rates)
print('TimeBins rate uncertainties:', time_bins.rate_uncertainty)
print('TimeBins centroids:', time_bins.centroids)
print('TimeBins range:', time_bins.range)

TimeBins rates: [12.5 25.  75.  25.  12.5  0. ]
TimeBins rate uncertainties: [12.5        17.67766953 43.30127019 17.67766953 12.5         0.        ]
TimeBins centroids: [0.15  0.25  0.325 0.45  0.55  0.65 ]
TimeBins range: (0.1, 0.7)


Several operations can be performed, including taking slices of the `TimeBins` and merging multiple `TimeBins` using the `slice()` and `merge()` methods.

In [16]:
# return first new TimeBins object by slicing original TimeBins from 0.1 to 0.25
slice1 = time_bins.slice(0.1, 0.25)
print(slice1)

<TimeBins: 2 bins;
 range (0.1, 0.3);
 1 contiguous segments>


In [17]:
# return second new TimeBins object by slicing original TimeBins from 0.5 to 0.8
slice2 = time_bins.slice(0.5, 0.8)
print(slice2)

<TimeBins: 2 bins;
 range (0.5, 0.7);
 1 contiguous segments>


In [18]:
# merge together first and second sliced TimeBins objects
merged = TimeBins.merge([slice1, slice2])
print(merged)

<TimeBins: 4 bins;
 range (0.1, 0.7);
 2 contiguous segments>


`TimeBins` can also be rebinned using a binning function. For example, we can rebin our `time_bins` by a factor of 3 using the `combine_by_factor` class from the core GDT toolkit.

In [19]:
from gdt.core.binning.binned import combine_by_factor

# rebin TimeBins object by combining bins by a factor of 3
rebinned = time_bins.rebin(combine_by_factor, 3)
print(rebinned)

<TimeBins: 2 bins;
 range (0.1, 0.7);
 2 contiguous segments>


Multiple `TimeBins` can be summed using the `sum()` method as long as they have the same bin edges. 

In [20]:
# sum original TimeBins object with itself
summed = TimeBins.sum([time_bins, time_bins])

# print counts of original TimeBins object, followed by counts of summed TimeBins object
print('Original TimeBins counts:', time_bins.counts)
print('Summed TimeBins counts:', summed.counts)

Original TimeBins counts: [1 2 3 2 1 0]
Summed TimeBins counts: [2. 4. 6. 4. 2. 0.]


An `EnergyBins` object can be created in the same way as a `TimeBins` object, by providing the counts, bin edges, and exposures for the `EnergyBins`. 

In [21]:
# create EnergyBins object with listed counts, edge boundaries, and exposures
counts = [1, 2, 3, 2, 1, 0]
low_edges = [10.0, 20.0, 40.0, 80.0, 160.0, 320.0]
high_edges = [20.0, 40.0, 80.0, 160.0, 320.0, 640.0]
exposure = 10.0
energy_bins = EnergyBins(counts, low_edges, high_edges, exposure)
print(energy_bins)

<EnergyBins: 6 bins;
 range (10.0, 640.0);
 1 contiguous segments>


Similarly, you can retrieve the same properties for `EnergyBins`, with some important differences. `EnergyBins` is defined to account for a spectrum that spans orders of magnitude, and therefore the bin centroids are calculated as the geometric mean, instead of the arithmetic mean. Also, the we will be accessing the differential rate and rate uncertainty (i.e. divided by the bin width) for this example with the `rates_per_kev` method, as opposed to the normal count rate.

In [22]:
# return centroids and differential count rates of EnergyBins object
print('EnergyBins centroids:', energy_bins.centroids)
print('EnergyBins differential count rates:', energy_bins.rates_per_kev)

EnergyBins centroids: [ 14.14213562  28.28427125  56.56854249 113.13708499 226.27416998
 452.54833996]
EnergyBins differential count rates: [0.01     0.01     0.0075   0.0025   0.000625 0.      ]


Like `TimeBins`, you can sum multiple `EnergyBins` together as long as they have the same energy range. 

In [23]:
# sum original EnergyBins object with itself
energy_bins2 = EnergyBins.sum([energy_bins, energy_bins])

# print counts of original EnergyBins object, followed by counts of summed EnergyBins object
print('Original EnergyBins counts:', energy_bins.counts)
print('Summed EnergyBins counts:', energy_bins2.counts)

Original EnergyBins counts: [1 2 3 2 1 0]
Summed EnergyBins counts: [2. 4. 6. 4. 2. 0.]


Maybe we don’t want all of our data for an analysis, but only a snippet. We can make a slice with the `slice()` method. 

In [24]:
# return new EnergyBins object by slicing original EnergyBins from 50.0 to 200.0
slice_energybins = energy_bins.slice(50.0, 200.0)

# print range of sliced EnergyBins object
print('Sliced EnergyBins range:', slice_energybins.range)

Sliced EnergyBins range: (40.0, 320.0)


We can also rebin the `EnergyBins` data using a binning algorithm. Here we will use the `combine_by_factor` class again, this time rebinning our data by a factor of 2. 

In [25]:
from gdt.core.binning.binned import combine_by_factor

# rebin EnergyBins object by combining bins by a factor of 2
rebinned_energybins = energy_bins.rebin(combine_by_factor, 2)

# return centroids and counts of rebinned EnergyBins object
print('Rebinned EnergyBins centroids:', rebinned_energybins.centroids)
print('Rebinned EnergyBins counts:', rebinned_energybins.counts)

Rebinned EnergyBins centroids: [ 20.  80. 320.]
Rebinned EnergyBins counts: [3 5 1]


In the case were we don’t have an energy calibration and instead only have counts in a series of energy channels, we use a `ChannelBins` object. All we need to create a `ChannelBins` object is a list of counts, energy channel numbers, and exposures. 

In [26]:
from gdt.core.data_primitives import ChannelBins

# create EnergyBins object with listed counts, channel numbers, and exposures
counts = [1, 2, 3, 2, 1, 0]
chan_nums = [0, 1, 2, 3, 4, 5]
exposure = 10.0
channel_bins = ChannelBins.create(counts, chan_nums, exposure)
print(channel_bins)

<ChannelBins: 6 bins;
 range (0, 5);
 1 contiguous segments>


The concept of bin edges is replaced by a channel number, therefore, we can still perform slicing and rebinning in the same way we do with `EnergyBins`.

In [27]:
# return new ChannelBins object by slicing original ChannelBins from 1 to 3
print(channel_bins.slice(1, 3))

<ChannelBins: 3 bins;
 range (1, 3);
 1 contiguous segments>


In [28]:
# rebin ChannelBins object by combining bins by a factor of 3
rebinned_channel_bins = channel_bins.rebin(combine_by_factor, 3)
print(rebinned_channel_bins)

<ChannelBins: 2 bins;
 range (0, 3);
 1 contiguous segments>


One behavior you should take note in rebinning `ChannelBins` is that the channel numbers are not renumbered.

In [29]:
# return channel numbers and number of bins for rebinned ChannelBins object
print('Rebinned ChannelBins channel numbers:', rebinned_channel_bins.chan_nums)
print('Rebinned ChannelBins number of bins:', rebinned_channel_bins.size)

Rebinned ChannelBins channel numbers: [0 3]
Rebinned ChannelBins number of bins: 2


## 2D Binned Data

We extend the philosophy of 1D time or energy data to 2D time and energy data in the `TimeChannelBins` and `TimeEnergyBins` classes. These data type are essentially 2D histograms, where one axis is time, and the other is energy, and the corresponding attributes and methods are available for the respective axes. Additionally, this class has the ability to integrate along either axis to produce a `TimeBins` or `ChannelBins`/`EnergyBins`.

To create a `TimeEnergyBins` object, you need a 2D array of counts (the number of counts in each time bin for each energy channel), the time bin edges, exposure in each bin, and energy bin edges. In general, many of the same properties and methods for `TimeBins` and `EnergyBins` are available here, keeping in mind that there are different methods for each axis.

In [30]:
from gdt.core.data_primitives import TimeEnergyBins

# create TimeEnergyBins object with listed counts, time bin edges, exposures, and energy bins edges
counts = [[1, 4, 1], [2, 3, 0], [3, 2, 1], [4, 1, 2]]
tstart = [0.1, 0.2, 0.3, 0.4]
tstop = [0.2, 0.3, 0.4, 0.5]
exposure = [0.8, 0.8, 0.8, 0.8]
emin = [10.0, 20.0, 40.0]
emax = [20.0, 40.0, 80.0]
te_bins = TimeEnergyBins(counts, tstart, tstop, exposure, emin, emax)
print(te_bins)

<TimeEnergyBins: 4 time bins;
 time range (0.1, 0.5);
 1 time segments;
 3 energy bins;
 energy range (10.0, 80.0);
 1 energy segments>


Attributes similar to those for the 1D counterparts of this data type are available, keeping in mind that there are different methods for each axis. Similar to the 1D counterparts, we can access various properties like the bin centroids:

In [31]:
# print energy centroids and time centroids for TimeEnergyBins object
print('TimeEnergyBins energy centroids:', te_bins.energy_centroids)
print('TimeEnergyBins time centroids:', te_bins.time_centroids)

TimeEnergyBins energy centroids: [14.14213562 28.28427125 56.56854249]
TimeEnergyBins time centroids: [0.15 0.25 0.35 0.45]


Or the number of time bins and energy channels:

In [32]:
# print number of time bins and number of energy channels for TimeEnergyBins object
print('TimeEnergyBins number of time bins:', te_bins.num_times)
print('TimeEnergyBins number of energy channels:', te_bins.num_chans)

TimeEnergyBins number of time bins: 4
TimeEnergyBins number of energy channels: 3


Or the bin widths:

In [33]:
# return time bin widths for TimeEnergyBins object
print('TimeEnergyBins time bin widths:', te_bins.time_widths)

# return energy bin widths for TimeEnergyBins object
print('TimeEnergyBins energy bin widths:', te_bins.chan_widths)

TimeEnergyBins time bin widths: [0.1 0.1 0.1 0.1]
TimeEnergyBins energy bin widths: [10. 20. 40.]


And we can immediately return the rates and uncertainty for each bin:

In [34]:
# return rates and rate uncertainties for TimeEnergyBins object
print('TimeEnergyBins rates:', te_bins.rates)
print('TimeEnergyBins rate uncertainties:', te_bins.rate_uncertainty)

TimeEnergyBins rates: [[1.25 5.   1.25]
 [2.5  3.75 0.  ]
 [3.75 2.5  1.25]
 [5.   1.25 2.5 ]]
TimeEnergyBins rate uncertainties: [[1.25       2.5        1.25      ]
 [1.76776695 2.16506351 0.        ]
 [2.16506351 1.76776695 1.25      ]
 [2.5        1.25       1.76776695]]


As for the available methods, similar to the 1D counterparts, you can retrieve a list of contiguous bins (`contiguous_time_bins()` and `contiguous_energy_bins()`), perform a slice (`slice_time()` and `slice_energy()`), rebin (`rebin_time()` and `rebin_energy()`), and merge multiple `TimeEnergyBins` across one axis (`merge_time()` and `merge_energy()`). In addition to these methods, you can also integrate along either axis of the `TimeEnergyBins` object to return the corresponding `TimeBins` or `EnergyBins` object:

In [35]:
# return TimeBins object by integrating TimeEnergyBins object over energy
time_bins_integrated = te_bins.integrate_energy()
print(time_bins_integrated)

<TimeBins: 4 bins;
 range (0.1, 0.5);
 1 contiguous segments>


In [36]:
# return EnergyBins object by integrating TimeEnergyBins object over time
energy_bins_integrated = te_bins.integrate_time()
print(energy_bins_integrated)

<EnergyBins: 3 bins;
 range (10.0, 80.0);
 1 contiguous segments>


`TimeChannelBins` objects are initialized in the same way as `TimeEnergyBins`, but instead of energy bin edges, the object is initialized with channel numbers.

In [37]:
from gdt.core.data_primitives import TimeChannelBins

# create TimeEnergyBins object with listed counts, time bin edges, exposures, and channel numbers
counts = [[1, 4, 1], [2, 3, 0], [3, 2, 1], [4, 1, 2]]
tstart = [0.1, 0.2, 0.3, 0.4]
tstop = [0.2, 0.3, 0.4, 0.5]
exposure = [0.8, 0.8, 0.8, 0.8]
chan_nums = [0, 1, 2]
tc_bins = TimeChannelBins(counts, tstart, tstop, exposure, chan_nums)
print(tc_bins)

<TimeChannelBins: 4 time bins;
 time range (0.1, 0.5);
 1 time segments;
 3 channels;
 channel range (0, 2);
 1 channel segments>


Most attributes and methods are the same between `TimeChannelBins` and `TimeEnergyBins`, with the exception of some of the attributes associated with the energy axis. We can integrate over either axis, noting that integrating over time will return a `ChannelBins` object.

In [38]:
# return TimeBins object by integrating TimeChannelBins object over channel
print(tc_bins.integrate_channels(chan_min=1, chan_max=3))

<TimeBins: 4 bins;
 range (0.1, 0.5);
 1 contiguous segments>


In [39]:
# return ChannelBins object by integrating TimeChannelBins object over time
print(tc_bins.integrate_time())

<ChannelBins: 3 bins;
 range (0, 2);
 1 contiguous segments>


We can rebin the energy channels, in this example using the `combine_by_factor` class to rebin the channels by a factor of 2. 

In [40]:
from gdt.core.binning.binned import combine_by_factor

# rebin TimeChannelBins object by combining bins by a factor of 2
print(tc_bins.rebin_channels(combine_by_factor, 2))

<TimeChannelBins: 4 time bins;
 time range (0.1, 0.5);
 1 time segments;
 1 channels;
 channel range (0, 0);
 1 channel segments>


We can slice by energy channel using the `slice_channels()` method. 

In [41]:
# return new TimeChannelBins object by slicing original TimeChannelBins from channel 1 to channel 3
print(tc_bins.slice_channels(1, 3))

<TimeChannelBins: 4 time bins;
 time range (0.1, 0.5);
 1 time segments;
 2 channels;
 channel range (1, 2);
 1 channel segments>


`TimeChannelBins` has the additional function (the `apply_bounds()` method) that allows us to apply an energy calibration via an `Ebounds` object and return a `TimeEnergyBins` object.

In [44]:
from gdt.core.data_primitives import Ebounds

# create Ebounds object from list of energy range minimums and maximums
emin = [10.0, 20.0, 40.0]
emax = [20.0, 40.0, 80.0]
ebounds = Ebounds.from_bounds(emin, emax)

# calibrate TimeChannelBins object using Ebounds object and return TimeEnergyBins object
te_bins_ebounds = tc_bins.apply_ebounds(ebounds)
print(te_bins_ebounds)

<TimeEnergyBins: 4 time bins;
 time range (0.1, 0.5);
 1 time segments;
 3 energy bins;
 energy range (10.0, 80.0);
 1 energy segments>


## Event Data

Instead of data binned in time, individual counts or photons may be recorded. These individual events have at least a time and energy attribute. Data like this are represented by an `EventList`. Typically these data are consider pseudo-unbinned, since they are truly unbinned in time (down to the precision of the instrument being able to measure the individual arrival times of photons), but are typically associated with binned energy channels. Similar to the other data types, the `EventList` has a variety of attributes and methods, including a `bin()` method that bins the `EventList` in time to produce a `TimeEnergyBins` object. 

To create an `EventList`, an array of event timestamps is required, as well as a corresponding energy channel number for each event. Optionally, an `Ebounds` representing the channel-number-to-energy mapping can be provided.

In [45]:
from gdt.core.data_primitives import EventList, Ebounds

# create EventList from random times and channels, and listed energy bounds of energy channels
times = np.random.exponential(1.0, size=100).cumsum()
channels = np.random.randint(0, 4, size=100)
ebounds = Ebounds.from_bounds([10.0, 20.0, 40.0, 80.0], [20.0, 40.0, 80.0, 160.0])
event_list = EventList(times, channels, ebounds=ebounds)
print(event_list)

<EventList: 100 events;
 time range (0.246242736042678, 107.7687909169182);
 channel range (0, 3)>


In this example, we generated 100 random times from a Poisson rate of 1 count/sec, and generated random energy channels from a 4-channel spectrum.

We can calculate the exposure of the `EventList` for the entire list or for a time range. Also, if there as in event deadtime, we can account for that by setting the `event_deadtime` parameter.

In [46]:
# calculate total exposure of EventList with deadtime per bin of 2.6e-6 seconds
print('EventList total exposure:', event_list.get_exposure(event_deadtime=2.6e-6))

# calculate exposure of EventList from 20.0 to 30.0 seconds with deadtime per bin of 2.6e-6 seconds
print('EventList exposure from 20 to 30 seconds:', event_list.get_exposure(time_ranges=[20.0, 30.0], event_deadtime=2.6e-6))

EventList total exposure: 107.52236358087553
EventList exposure from 20 to 30 seconds: 9.9999844


We can also easily access multiple properties of the `EventList`, such as the size, time range, and energy range. 

In [47]:
# print number of events, range of times, and energy range of channels for EventList
print('EventList number of events:', event_list.size)
print('EventList time range:', event_list.time_range)
print('EventList energy range:', event_list.energy_range)

EventList number of events: 100
EventList time range: (0.246242736042678, 107.7687909169182)
EventList energy range: (10.0, 160.0)


The `EventList` can be sliced in time, by channel, or energy range (if there is an associated `Ebounds`), which returns a new `EventList` containing the slice.

In [48]:
# return new EventList by slicing original EventList in time from 20.0 to 30.0
time_sliced = event_list.time_slice(20.0, 30.0)
print(time_sliced)

<EventList: 11 events;
 time range (20.081797911314982, 29.96485378051192);
 channel range (0, 3)>


In [49]:
# return new EventList by slicing original EventList by channel from 0 to 1
channel_sliced = event_list.channel_slice(0, 1)
print(channel_sliced)

<EventList: 49 events;
 time range (0.246242736042678, 106.21118353189976);
 channel range (0, 1)>


In [50]:
# return new EventList by slicing original EventList in energy from 50.0 to 150.0
energy_sliced = event_list.energy_slice(50.0, 150.0)
print(energy_sliced)

<EventList: 51 events;
 time range (2.5422067327550657, 107.7687909169182);
 channel range (2, 3)>


Multiple `EventLists` can be merged into a single list using the `merge()` method. If there is a chance that the `EventLists` may contain duplicate events, you may want to set the `force_unique` keyword.

In [51]:
# merge channel sliced and energy sliced EventList objects together into one EventList
merged_eventlist = EventList.merge([channel_sliced, energy_sliced], force_unique=True)
print(merged_eventlist)

<EventList: 100 events;
 time range (0.246242736042678, 107.7687909169182);
 channel range (0, 3)>


Given an algorithm to bin the event data, we can convert the `EventList` to a `TimeEnergyBins`. Here, we will use the `bin_by_time` class to bin our data.

In [52]:
from gdt.core.binning.unbinned import bin_by_time

# bin EventList using the bin_by_time algorithm and return a TimeEnergyBins object
te_bins_binned = event_list.bin(bin_by_time, 10.0, event_deadtime=2.6e-6)
print(te_bins_binned)

<TimeEnergyBins: 11 time bins;
 time range (0.246242736042678, 110.24624273604267);
 1 time segments;
 4 energy bins;
 energy range (10.0, 160.0);
 1 energy segments>


Even without converting to a `TimeEnergyBins`, we can readily extract the count spectrum from the `EventList` using the `count_spectrum()` method. 

In [53]:
# extract the count spectrum as an EnergyBins object from the EventList
count_spec = event_list.count_spectrum(event_deadtime=2.6e-6)
print(count_spec)

<EnergyBins: 4 bins;
 range (10.0, 160.0);
 1 contiguous segments>


Note that if the energy calibration (ebounds) is not set for `EventList`, the `bin()` method will return a `TimeChannelBins` object and the `count_spectrum()` methods returns a `ChannelBins` object.

## Response Matrix

A response matrix is a matrix that represents a detector energy response. Typically, a detector’s response will modify the incident spectrum so that the recorded spectrum is a convolution of the true incident spectrum with the detector response. Many factors may go into the calculation of the response matrix, such as energy dispersion, photon scattering, and absorption. Additionally, the magnitude of the response matrix is the effective area of the detector as a function of energy, accounting for factors such as the quantum efficiency of the detector. Often the response matrix contains significant off-diagonal contributions, which prevents matrix inversion, therefore estimates of the incident spectrum must be made indirectly via forward-folding techniques.

To create a `ResponseMatrix` object, we need to provide a 2D array of effective areas that correspond to the number of incident photon bins and the number of output energy channels. We also need to provide the incident photon bin edges and the output energy channel edges. In our simple example, we will create a simple diagonal response where the input and output edges are the same.

In [54]:
from gdt.core.data_primitives import ResponseMatrix

# create a ResponseMatrix from listed 2D matrix, photon bin edges, and energy channel edges
matrix = np.diag([0.1, 0.2, 0.3, 0.2, 0.1, 0.1])
emin = [10., 20., 40., 80., 160., 320.]
emax = [20., 40., 80., 160., 320., 640.]
chanlo = emin
chanhi = emax
rsp = ResponseMatrix(matrix, emin, emax, chanlo, chanhi)
print(rsp)

<ResponseMatrix: 6 energy bins; 6 channels>


Similar to the other data primitives, there are several attributes provided. For example, we can easily obtain the widths of the incident photon bins or the centroids of the output energy bins like so:

In [55]:
# return centroids of energy channels for ResponseMatrix
print('ResponseMatrix energy bin centroids:', rsp.channel_centroids)

# return widths of photon bins for ResponseMatrix
print('ResposneMatrix photon bin widths:', rsp.photon_bin_widths)

ResponseMatrix energy bin centroids: [ 14.14213562  28.28427125  56.56854249 113.13708499 226.27416998
 452.54833996]
ResposneMatrix photon bin widths: [ 10.  20.  40.  80. 160. 320.]


The effective area can be estimated at any arbitrary energy within the bounds of the response using the `effective_area()` method. 

In [56]:
# calculate effective area at energy value 50.0 for ResponseMatrix
print('ResponseMatrix effective area at energy 50.0:', rsp.effective_area(50.0))

# calculate effective area at energy value 135.0 for ResponseMatrix
print('ResponseMatrix effective area at energy 135.0:', rsp.effective_area(135.0))

ResponseMatrix effective area at energy 50.0: 0.2710806010829534
ResponseMatrix effective area at energy 135.0: 0.12418578120734841


Most importantly, any photon model can be folded through the response using the `fold_spectrum()` method to produce the detector-convolved count spectrum. While you can certainly write a simple photon model function to fold through the response, there are built-in models in the spectral functions module. In our example, we will fold a power law model from the Core GDT toolkit's `PowerLaw` class through our response.

In [57]:
from gdt.core.spectra.functions import PowerLaw

# initialize power law function
pl = PowerLaw()

# fold power law photon spectrum through ResponseMatrix to return a count spectrum
count_spectrum = rsp.fold_spectrum(pl.fit_eval, (0.1, -2.2))
print(count_spectrum)

[7.39378818 6.43666647 4.20258271 1.21952025 0.26541351 0.11552794]


Thus, we have produced the count spectrum in counts/second.

The response can also be rebinned along the channel axis using the `rebin()` method, if desired. This may be useful in cases where the corresponding spectral data is rebinned, although keep in mind that rebinning causes loss of information. To rebin the response, we specify either the factor at which to combine bins or a list of bin edge indices defining the bin edges that will remain after rebinning.

In [58]:
# rebin ResponseMatrix by a factor of 2
rebinned_factor = rsp.rebin(factor=2)
print(rebinned_factor)

<ResponseMatrix: 6 energy bins; 3 channels>


In [59]:
# rebin ResponseMatrix with listed bin edge indices
rebinned_edges = rsp.rebin(edge_indices=[0, 1, 5])
print(rebinned_edges)

<ResponseMatrix: 6 energy bins; 2 channels>


Finally, the input energy axis of the response can be resampled using the `resample()` method to increase or decrease the photon energy resolution or redefine the input photon bin edges. Keep in mind that this may introduce a systematic error via interpolation of the response, so you should always check to make sure the response behaves as expected after resampling. To resample, you can either specify the number of photon bins you want, in which case the bin edges are spaced logarithmically, or you can specify the edges of the bins. The photon bins can only be resampled within the energy bounds of the original response.

In [60]:
# resample the input energy axis of ResposneMatrix to have 12 photon bins
resampled_num = rsp.resample(num_photon_bins=12)
print(resampled_num)

<ResponseMatrix: 12 energy bins; 6 channels>


In [61]:
# resample the input energy axis of ResponseMatrix to have the listed bin edges
resampled_edges = rsp.resample(photon_bin_edges=(10., 30., 90., 640.))
print(resampled_edges)

<ResponseMatrix: 3 energy bins; 6 channels>


## Parameter

A parameter in the context of the GDT is a variable that has an associated uncertainty regarding its value, and usually results from a fit to data. The `Parameter` class is a simple container for a value and the associated uncertainty and provides a few functions and different representations of the parameter.

To create a `Parameter` object, we need to provide at least a value and the 1-sigma equivalent uncertainty. We can also provide other useful information, such as the name of the parameter, it’s units, and support over which it is valid.

In [62]:
from gdt.core.data_primitives import Parameter

# create Parameter with value of 100.0, symmetrical uncertainty of 5.0, and listed name
param1 = Parameter(100.0, 5.0, name='param1')
print(param1)

param1: 100.00 +/- 5.00


In [63]:
# create Parameter with value of 100.0, asymmetrical uncertainty of (-5.0, +10.0), listed name and units, and valid support from 0 to infinity
param_epeak = Parameter(100.0, (5.0, 10.0), name='Epeak', units='keV', support=(0.0, np.inf))
print(param_epeak)


Epeak: 100.00 +10.00/-5.00 keV


In the first example, we assigned a symmetric 1-sigma uncertainty (5.0) to the parameter value, and in the second example, we assigned an asymmetric uncertainty. Once initialized, we can easily access multiple attributes like so:

In [64]:
# print name of Parameter
print('Parameter name:', param_epeak.name)

Parameter name: Epeak


In [65]:
# print support range over which Parameter is valid
print('Parameter support range:', param_epeak.support)

Parameter support range: (0.0, inf)


In [66]:
# print uncertainty of Parameter
print('Parameter uncertainty:', param_epeak.uncertainty)

Parameter uncertainty: (5.0, 10.0)


In [67]:
# print units of Parameter
print('Parameter units:', param_epeak.units)

Parameter units: keV


There are some other convenience functions, such as checking if the value is within the valid support using the `valid_value()` method.

In [68]:
# check if Parameter value is allowed within Parameter range
print('Parameter is valid within support range:', param_epeak.valid_value())

Parameter is valid within support range: True


Additionally, the 1-sigma range can be returned using the `one_sigma_range()` method. 

In [69]:
# print 1-sigma range for Parameter
print('Parameter 1-sigma range:', param_epeak.one_sigma_range())

Parameter 1-sigma range: (95.0, 110.0)


And finally, the parameter can be represented in the common FITS format by the `to_fits_value()` method. 

In [70]:
# print Parameter in common FITS format
print('Parameter in FITS format:', param_epeak.to_fits_value())

Parameter in FITS format: (100.0, 10.0, 5.0)


Congratulations, you've completed the lesson on data primitives!